# Autocorrelation analysis of Berlin by median price of airbnbs/neighborhood

In [48]:
import esda
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import libpysal as lp
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point


from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd


In [49]:
ref = pd.read_csv(
    "../data - AirBnb/listings.csv.gz", compression='gzip', on_bad_lines='skip')


ref['price'] = ref['price'].str.replace('$', '').str.replace(',', '').astype(float)
ref = ref[ref['price'] < 1000]

median = ref['price'].median()

ref = ref[['neighbourhood_cleansed', 'price', 'host_is_superhost', 'property_type', 'room_type', 'accommodates', 'beds',
             'instant_bookable', 'minimum_nights', 'maximum_nights', 'bathrooms_text']]

ref['host_is_superhost'] = ref['host_is_superhost'].map({'t': True, 'f': False})
ref['instant_bookable'] = ref['instant_bookable'].map({'t': True, 'f': False})

# ref['shared_bath'] = ref['bathrooms_text'].apply(lambda x: True if 'shared' in str(x).lower() else False)
ref['num_bathrooms'] = ref['bathrooms_text'].str.extract('(\d+)').astype(float)
ref['num_bathrooms'] = ref['num_bathrooms'].fillna(0)

ref.drop('bathrooms_text', axis=1, inplace=True)


<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_19137/2671716690.py:17: SyntaxWarning: invalid escape sequence '\d'
  ref['num_bathrooms'] = ref['bathrooms_text'].str.extract('(\d+)').astype(float)


In [50]:
ref['num_bathrooms'].value_counts()

num_bathrooms
1.0    8348
2.0     858
0.0      65
3.0      63
4.0      12
8.0       3
5.0       2
6.0       2
Name: count, dtype: int64

In [51]:
ref.head(20)

,neighbourhood_cleansed,price,host_is_superhost,property_type,room_type,accommodates,beds,instant_bookable,minimum_nights,maximum_nights,shared_bath,num_bathrooms
0,Prenzlauer Berg Südwest,83.0,False,Entire rental unit,Entire home/apt,4,2.0,False,63,184,False,1.0
1,Prenzlauer Berg Südwest,180.0,False,Entire rental unit,Entire home/apt,7,7.0,False,6,14,False,2.0
2,Neue Kantstraße,116.0,False,Entire rental unit,Entire home/apt,4,2.0,False,2,365,False,1.0
3,Prenzlauer Berg Nordwest,70.0,False,Entire rental unit,Entire home/apt,1,1.0,False,150,1125,False,1.0
4,Zehlendorf Nord,100.0,False,Entire rental unit,Entire home/apt,2,2.0,False,183,730,False,1.0
5,nördliche Luisenstadt,90.0,False,Entire condo,Entire home/apt,4,1.0,False,93,365,False,1.0
6,Schöneberg-Nord,45.0,False,Entire rental unit,Entire home/apt,2,2.0,False,93,365,False,1.0
8,Reuterstraße,25.0,True,Entire rental unit,Entire home/apt,2,1.0,False,92,365,False,1.0
9,Prenzlauer Berg Ost,120.0,False,Entire townhouse,Entire home/apt,4,2.0,False,3,365,False,1.0
10,Prenzlauer Berg Südwest,179.0,True,Entire loft,Entire home/apt,4,2.0,False,3,30,False,1.0


In [52]:
ref.isna().sum()

neighbourhood_cleansed      0
price                       0
host_is_superhost          36
property_type               0
room_type                   0
accommodates                0
beds                      143
instant_bookable            0
minimum_nights              0
maximum_nights              0
shared_bath                 0
num_bathrooms               0
dtype: int64

In [53]:
ref['host_is_superhost'] = ref['host_is_superhost'].fillna(False)
ref['beds'] = ref['beds'].fillna(1)

In [54]:
categories=ref['neighbourhood_cleansed'].unique().tolist()

In [55]:


X = ref.drop('price', axis=1)
y = ref['price']

numeric_features = ['accommodates', 'beds', 'minimum_nights', 'maximum_nights', 'num_bathrooms']
categorical_features = ['neighbourhood_cleansed', 'property_type', 'room_type']

numeric_transformer = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', ohe, categorical_features)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [ 10, 20, 30],
    'regressor__min_samples_leaf': [1, 2, 3],
    'regressor__min_samples_split': [4, 6, 7, 8, 10]
}

# param_grid = {
#     'regressor__n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 21],
# }

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, error_score='raise')
grid_search.fit(X_train, y_train)

grid_search.best_params_



Fitting 5 folds for each of 135 candidates, totalling 675 fits


[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=4, regressor__n_estimators=100; total time=   3.6s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=4, regressor__n_estimators=100; total time=   3.9s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=4, regressor__n_estimators=100; total time=   4.0s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=4, regressor__n_estimators=100; total time=   4.1s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=4, regressor__n_estimators=100; total time=   4.3s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=4, regressor__n_estimators=200; total time=   6.0s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__min_samples_split=4, regressor__n_estimators=200; tota

{'regressor__max_depth': 30,
 'regressor__min_samples_leaf': 1,
 'regressor__min_samples_split': 4,
 'regressor__n_estimators': 200}

In [60]:
grid_search.best_score_

0.44440411181738726

In [61]:
rf_best = grid_search.best_estimator_
rf_best.fit(X_train, y_train)
rf_best.score(X_train, y_train)

0.8265013478940005

In [58]:
# get estimator from pipeline
rf = grid_search.best_estimator_.named_steps['regressor']

In [59]:
ohe = ohe.fit(X_train[categorical_features])
feature_names_after_encoding = ohe.get_feature_names_out(categorical_features)

# Concatenate numeric features and one-hot-encoded features
all_feature_names = numeric_features + list(feature_names_after_encoding)

# Get feature importances
feature_importances = rf.feature_importances_

# Print or use the feature importances as needed
importances = {}
for feature, importance in zip(all_feature_names, feature_importances):
    importances[feature] = importance

final_importances = {}
for cat in categorical_features:
    for key,value in importances.items():
        if key.startswith(cat):
            final_importances[cat] = final_importances.get(cat, 0) + value

for cat in numeric_features:
    final_importances[cat] = importances[cat]

final_importances

{'neighbourhood_cleansed': 0.26167705896429405,
 'property_type': 0.13136262045697408,
 'room_type': 0.020172783307021817,
 'shared_bath': 0.01992099543924666,
 'accommodates': 0.25218532101982083,
 'beds': 0.03980371665268648,
 'minimum_nights': 0.10037210372631526,
 'maximum_nights': 0.06623597204969117,
 'num_bathrooms': 0.1082694283839497}